# Tier C - The Transformer

This uses DistilBERT, a lighter transformer model. Transformers use self-attention. This is a mechanism that lets models understand which words matter most in context. For example, in the phrase *"The girl and her brother"*, the transformer associates *her* with *The girl* rather than treating each word independently. This context-awareness could help it pick up on the subtle stylistic patterns we identified in Task 1.

In [1]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    TrainingArguments, 
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
import evaluate
import glob
import os
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split

print("Hello")

MODEL_ID = "distilbert-base-uncased"
DATASET_DIR = Path('../dataset')
LR = 2e-4
BATCH_SIZE = 16
EPOCHS = 3

def load_texts_from_directory(directory_path, class_label):
    data = []
    txt_files = glob.glob(os.path.join(str(directory_path), '*.txt'))
    
    for file_path in tqdm(txt_files, desc=f"  Loading {class_label}"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
            if text:
                data.append({
                    'text': text,
                    'label': class_label,
                    'file_name': os.path.basename(file_path)
                })
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
    
    return data

print("\nLoading Class 1 (Human-written)...")
class1_data = []
for author in ['01-arthur-conan-doyle', '02-pg-wodehouse', '03-mark-twain', '04-william-shakespeare']:
    path = DATASET_DIR / 'class1-human-written' / author / 'extracted_paragraphs'
    class1_data.extend(load_texts_from_directory(path, 0))

print("\nLoading Class 2 (AI-written)...")
class2_path = DATASET_DIR / 'class2-ai-written' / 'ai-generated-paragraphs'
class2_data = load_texts_from_directory(class2_path, 1)

print("\nLoading Class 3 (AI-mimicry)...")
class3_data = []
for author in ['01-arthur-conan-doyle', '02-pg-wodehouse', '03-mark-twain', '04-william-shakespeare']:
    path = DATASET_DIR / 'class3-ai-mimicry' / author
    class3_data.extend(load_texts_from_directory(path, 2))

all_data = class1_data + class2_data + class3_data
df = pd.DataFrame(all_data)

print(f"\nDataset loaded: {len(df)} total samples")
print(f"  Class 1 (Human): {len(class1_data)}")
print(f"  Class 2 (AI): {len(class2_data)}")
print(f"  Class 3 (AI-mimicry): {len(class3_data)}")

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test":  Dataset.from_pandas(test_df)
})

print(f"\nLoading Tokenizer for {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)

tokenized_datasets = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Loading Base Model...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID, num_labels=3
)

print("Applying LoRA (Low-Rank Adaptation)...")
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False, 
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./lora_checkpoints",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting Training (Tier C)...")
trainer.train()
print("\nFinal Evaluation on Test Set:")
eval_results = trainer.evaluate()
print(f"Accuracy: {eval_results['eval_accuracy']:.4f}")

model.save_pretrained("tier_c_final_model")
tokenizer.save_pretrained("tier_c_final_model")
print("Model and tokenizer saved to 'tier_c_final_model'")

/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hello

Loading Class 1 (Human-written)...


  Loading 0: 100%|██████████| 480/480 [00:00<00:00, 8466.39it/s]



Loading Class 2 (AI-written)...


  Loading 1: 100%|██████████| 988/988 [00:00<00:00, 7640.25it/s]


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.016221,0.996178
2,No log,0.010092,0.997452
3,0.089741,0.007909,0.998726


/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)



Final Evaluation on Test Set:


/home/samarth/Documents/precog-task/human-or-ai/venv2/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Accuracy: 0.9987
Model and tokenizer saved to 'tier_c_final_model'


# Results

It is performing extremely well. Suspiciously well. On first glance, I wonder whether or not it's overfitting...

However, I have a hypothesis about this which I will now test. 

**My Hypothesis:** I think this is genuine, given the vast mathematical and semantic differences in the dataset. Transformers should be able to perform significantly better than the vector space embeddings of the semanticist as well as the pure mathematical state approach of the statistician...

Our semanticist was already able to correctly classify with an accuracy of 96%. Now ontop of that we give it the function of attention. [*Attention is all you need,*](https://arxiv.org/pdf/1706.03762) so it would make sense that with attention, the model is able to perform to a significantly higher level of accuracy.

I argue that the fact of the matter is that AI is currently neither writes in the same style as any of our selected authors, nor can it accurately mimic the semantic details of any of our authors.

**Now, I mentioned that we'd test this.** My plan here is to do a sanity test... 